In [6]:
import numpy as np

class RandomForest:
    def __init__(self, n_trees=10, max_depth=None, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            # Create a random subset of the training data
            indices = np.random.choice(len(X), size=len(X), replace=True)
            subset_X = X[indices]
            subset_y = y[indices]

            # Create a decision tree and fit it to the random subset
            tree = DecisionTree(max_depth=self.max_depth, max_features=self.max_features)
            tree.fit(subset_X, subset_y)
            self.trees.append(tree)

    def predict(self, X):
        # Make predictions for each tree and aggregate the results
        tree_predictions = [tree.predict(X) for tree in self.trees]
        ensemble_predictions = np.array(tree_predictions).T  # Transpose for per-instance predictions
        final_predictions = [np.argmax(np.bincount(instance_predictions)) for instance_predictions in ensemble_predictions]
        return final_predictions

class DecisionTree:
    def __init__(self, max_depth=None, max_features=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.tree = None

    def calculate_entropy(self, y):
        unique_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def calculate_information_gain(self, X, y, feature_idx):
        # Calculate entropy before the split
        total_entropy = self.calculate_entropy(y)

        # Calculate entropy after the split
        unique_values = np.unique(X[:, feature_idx])
        weighted_entropy = 0

        for value in unique_values:
            subset_y = y[X[:, feature_idx] == value]
            weighted_entropy += (len(subset_y) / len(y)) * self.calculate_entropy(subset_y)

        information_gain = total_entropy - weighted_entropy
        return information_gain

    def find_best_split(self, X, y, features):
        if self.max_features:
            features = np.random.choice(features, size=self.max_features, replace=False)

        best_feature = None
        best_information_gain = -1

        for feature_idx in features:
            information_gain = self.calculate_information_gain(X, y, feature_idx)

            if information_gain > best_information_gain:
                best_feature = feature_idx
                best_information_gain = information_gain

        return best_feature

    def fit(self, X, y, depth=0, features=None):
        if features is None:
            features = list(range(X.shape[1]))

        # Check termination conditions
        if len(np.unique(y)) == 1:
            # Pure node, all samples have the same class
            self.tree = {'class': y[0]}
            return

        if self.max_depth is not None and depth == self.max_depth:
            # Maximum depth reached
            self.tree = {'class': np.argmax(np.bincount(y))}
            return

        if not features:
            # No more features to split on
            self.tree = {'class': np.argmax(np.bincount(y))}
            return

        # Find the best feature to split on
        best_feature = self.find_best_split(X, y, features)

        if best_feature is None:
            # Unable to find a split
            self.tree = {'class': np.argmax(np.bincount(y))}
            return

        # Split the data based on the best feature
        unique_values = np.unique(X[:, best_feature])
        self.tree = {'feature': best_feature, 'children': {}}

        for value in unique_values:
            # Create a child node for each unique value of the best feature
            subset_X = X[X[:, best_feature] == value]
            subset_y = y[X[:, best_feature] == value]
            subset_features = [f for f in features if f != best_feature]

            child_tree = DecisionTree(max_depth=self.max_depth, max_features=self.max_features)
            child_tree.fit(subset_X, subset_y, depth + 1, subset_features)

            self.tree['children'][value] = child_tree.tree

    def predict_instance(self, x, tree):
        if 'class' in tree:
            return tree['class']
        feature_value = x[tree['feature']]
        if feature_value not in tree['children']:
            return np.argmax(np.bincount(y))  # Predict the majority class
        return self.predict_instance(x, tree['children'][feature_value])

    def predict(self, X):
        return [self.predict_instance(x, self.tree) for x in X]

# Example usage:
X_train = np.array([[0, 1, 1, 0, 0],
                    [1, 0, 0, 1, 1],
                    [1, 1, 1, 0, 0],
                    [0, 0, 1, 1, 1],
                    [0, 1, 0, 0, 1],
                    [1, 0, 1, 1, 0],
                    [1, 1, 0, 0, 0],
                    [0, 0, 0, 1, 1],
                    [1, 0, 0, 1, 0],
                    [0, 1, 1, 0, 1]])
y_train = np.array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0])

# Create and train the Random Forest
forest = RandomForest(n_trees=5, max_depth=2, max_features=3)
forest.fit(X_train, y_train)

# Example prediction
X_test = np.array([[0, 1, 1, 0, 1],
                   [1, 0, 0, 1, 0]])
predictions = forest.predict(X_test)

print("Predictions:", predictions)

for i, tree in enumerate(forest.trees):
    print(f"Tree {i + 1}:\n{tree.tree}\n")

Predictions: [0, 1]
Tree 1:
{'feature': 2, 'children': {0: {'feature': 0, 'children': {0: {'class': 1}, 1: {'class': 1}}}, 1: {'class': 0}}}

Tree 2:
{'feature': 2, 'children': {0: {'feature': 0, 'children': {0: {'class': 0}, 1: {'class': 1}}}, 1: {'class': 0}}}

Tree 3:
{'feature': 0, 'children': {0: {'feature': 2, 'children': {0: {'class': 0}, 1: {'class': 0}}}, 1: {'feature': 2, 'children': {0: {'class': 1}, 1: {'class': 0}}}}}

Tree 4:
{'feature': 0, 'children': {0: {'feature': 2, 'children': {0: {'class': 0}, 1: {'class': 0}}}, 1: {'feature': 4, 'children': {0: {'class': 1}, 1: {'class': 1}}}}}

Tree 5:
{'feature': 0, 'children': {0: {'class': 0}, 1: {'feature': 1, 'children': {0: {'class': 0}, 1: {'class': 1}}}}}

